### Install some libs

!pip install treelib

!pip3 install tree_sitter

### Import some libs

In [30]:
from treelib import Node, Tree
from tree_sitter import Language, Parser
import os
import numpy as np

### Initialize Tree-sitter go parser

In [31]:
Language.build_library(
  # Store the library in the `build` directory
  'build/my-languages.so',

  # Include one or more languages
  [
    'tree-sitter-go'
  ]
)

GO_LANGUAGE = Language('build/my-languages.so', 'go')

parser = Parser()
parser.set_language(GO_LANGUAGE)

## Step I : Find the Feature Toggles (FT)

In [32]:
keywords = ["CAAValidationMethods", "CAAAccountURI", "EnforceMultiVA", "MultiVAFullResults", 
            "MandatoryPOSTAsGET", "AllowV1Registration", "V1DisableNewValidations", "StoreRevokerInfo", 
            "RestrictRSAKeySizes", "FasterNewOrdersRateLimit", "ECDSAForAll", "ServeRenewalInfo"]

### List files with keywords

#### List all .go files

In [33]:
go_folders = [x[0] for x in os.walk("./boulder/")]

go_files = []
for dir_name in go_folders:
    files = [dir_name+"/"+k for k in os.listdir(dir_name) if k[len(k)-3:] ==".go"]
    go_files.extend(files)

print("10 random .go files in this project :")
for k in range(10):
    print(go_files[np.random.randint(len(go_files))])

10 random .go files in this project :
./boulder/vendor/google.golang.org/protobuf/internal/impl/codec_field.go
./boulder/iana/iana_test.go
./boulder/vendor/golang.org/x/sys/unix/zsysnum_linux_sparc64.go
./boulder/vendor/golang.org/x/sys/unix/syscall_freebsd_arm.go
./boulder/vendor/golang.org/x/net/ipv6/zsys_aix_ppc64.go
./boulder/vendor/golang.org/x/sys/unix/unveil_openbsd.go
./boulder/test/load-generator/state.go
./boulder/vendor/google.golang.org/grpc/server.go
./boulder/vendor/go.opentelemetry.io/otel/attribute/iterator.go
./boulder/vendor/google.golang.org/appengine/timeout.go


In [34]:
len(go_files)

2143

#### List the interesting files, containing at least one keyword with the featureflag 

In [35]:
go_file_interests = []

for file in go_files:
    s = ""
    with open(file, "r") as f:
        s+=f.read()+"\n"
    kw_file = [k for k in keywords if k in s]
    if len(kw_file) > 0 and "features" in s:
        print(file, "contains :")
        for kw in kw_file:
            print("->",kw)
        go_file_interests.append(file)
        print("\n")

./boulder/wfe/wfe.go contains :
-> AllowV1Registration


./boulder/wfe/wfe_test.go contains :
-> AllowV1Registration


./boulder/features/features.go contains :
-> CAAValidationMethods
-> CAAAccountURI
-> EnforceMultiVA
-> MultiVAFullResults
-> MandatoryPOSTAsGET
-> AllowV1Registration
-> V1DisableNewValidations
-> StoreRevokerInfo
-> RestrictRSAKeySizes
-> FasterNewOrdersRateLimit
-> ECDSAForAll
-> ServeRenewalInfo


./boulder/features/featureflag_string.go contains :
-> CAAValidationMethods
-> CAAAccountURI
-> EnforceMultiVA
-> MultiVAFullResults
-> MandatoryPOSTAsGET
-> AllowV1Registration
-> V1DisableNewValidations
-> StoreRevokerInfo
-> RestrictRSAKeySizes
-> FasterNewOrdersRateLimit
-> ECDSAForAll
-> ServeRenewalInfo


./boulder/goodkey/good_key_test.go contains :
-> RestrictRSAKeySizes


./boulder/goodkey/good_key.go contains :
-> RestrictRSAKeySizes


./boulder/ca/ca.go contains :
-> ECDSAForAll


./boulder/ca/ca_test.go contains :
-> ECDSAForAll


./boulder/sa/sa.go contains :

In [36]:
len(go_file_interests)

18

## Step 3 - Search Feature Toggles dependencies in the AST

In [37]:
for gfi in go_file_interests:
    
    s = ""
    with open(gfi, "r") as f:
        s+=f.read()+"\n"

    print("File :            ", gfi, "\n")

    source = bytes(s, "utf8")
    ast = parser.parse(source)

    root_node = ast.root_node

    tree = Tree()

    root_node = ast.root_node

    type_nodes = dict()

    def get_name(node):
        return source[node.start_byte:node.end_byte].decode('utf8')

    def get_id(node):
        global type_nodes
        node_type = node.type
        if node_type not in type_nodes:
            type_nodes[node_type]=1
        else:
            type_nodes[node_type]+=1
        return node_type+str(type_nodes[node_type])

    def process(root_id, node):
        global tree
        node_id = get_id(node)
        node_name = get_name(node)
        for kw in keywords:
            if kw in node_name and node.type == 'if_statement':
                print("\n\n"+get_name(node)+"\n\n")
                for c in node.children:
                    if c.type == 'binary_expression' or c.type == 'unary_expression' or c.type == 'call_expression':
                        print("->",get_name(c))
        tree.create_node(node_name,
                         node_id,
                         parent = root_id)
        if len(node.children) != 0:
            for i in range(len(node.children)):
                process(node_id, node.children[i])

    tree.create_node("root", "root")
    for i in range(len(root_node.children)):
        process("root", root_node.children[i])
    print("\n")

    #tree.show()

File :             ./boulder/wfe/wfe.go 



if !features.Enabled(features.AllowV1Registration) {
		wfe.sendError(response, logEvent, probs.Unauthorized("Account creation on ACMEv1 is disabled. "+
			"Please upgrade your ACME client to a version that supports ACMEv2 / RFC 8555. "+
			"See https://community.letsencrypt.org/t/end-of-life-plan-for-acmev1/88430 for details."), nil)
		return
	}


-> !features.Enabled(features.AllowV1Registration)


File :             ./boulder/wfe/wfe_test.go 



File :             ./boulder/features/features.go 



File :             ./boulder/features/featureflag_string.go 



File :             ./boulder/goodkey/good_key_test.go 



File :             ./boulder/goodkey/good_key.go 



if features.Enabled(features.RestrictRSAKeySizes) {
		if !acceptableRSAKeySizes[modulusBitLen] {
			return badKey("key size not supported: %d", modulusBitLen)
		}
	} else {
		const maxKeySize = 4096
		if modulusBitLen < 2048 {
			return badKey("key too small: %d", modulusBit

In [22]:
type_nodes

{'package_clause': 1,
 'package': 1,
 'package_identifier': 71,
 '\n': 279,
 'import_declaration': 1,
 'import': 1,
 'import_spec_list': 1,
 '(': 199,
 'import_spec': 30,
 'interpreted_string_literal': 173,
 '"': 346,
 ')': 199,
 'var_declaration': 15,
 'var': 15,
 'var_spec': 15,
 'identifier': 628,
 '=': 30,
 'expression_list': 208,
 'function_declaration': 19,
 'func': 27,
 'parameter_list': 36,
 'parameter_declaration': 54,
 'type_identifier': 166,
 'pointer_type': 39,
 '*': 42,
 'qualified_type': 67,
 '.': 257,
 'block': 55,
 '{': 172,
 'short_var_declaration': 77,
 ',': 422,
 ':=': 81,
 'call_expression': 159,
 'selector_expression': 190,
 'field_identifier': 364,
 'argument_list': 159,
 'int_literal': 36,
 'return_statement': 11,
 'return': 11,
 '}': 172,
 'composite_literal': 60,
 'literal_value': 109,
 'keyed_element': 174,
 ':': 174,
 'slice_type': 22,
 '[': 48,
 ']': 48,
 'element': 93,
 'unary_expression': 18,
 '&': 14,
 'comment': 65,
 'assignment_statement': 9,
 'binary_e

In [14]:
# Pattern matching -> does not work yet

query = GO_LANGUAGE.query("""
(function_definition
  name: (identifier) @function.def)

(call
  function: (identifier) @function.call)
""")

captures = query.captures(tree.root_node)

